In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from Shelter import AnimalShelter


In [2]:
###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "Geese79"
host = 'nv-desktop-services.apporto.com'
port = 31049
database = 'AAC'
collections = 'animals'

# Connect to database via CRUD Module
db = AnimalShelter(username, password, host, port, database, collections)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read())

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

ServerSelectionTimeoutError: nv-desktop-services.apporto.com:31049: [Errno 11001] getaddrinfo failed (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 67af4fe77119b5ff1a00e951, topology_type: Unknown, servers: [<ServerDescription ('nv-desktop-services.apporto.com', 31049) server_type: Unknown, rtt: None, error=AutoReconnect('nv-desktop-services.apporto.com:31049: [Errno 11001] getaddrinfo failed (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [ ]:
#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)


image_filename = 'Grazioso Salvare Logo.png' # Added Grazioso logo
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    #Added unique identifier
    html.Center(html.B(html.H1('Grazioso Salvare Aniaml Database by Nathan Estrada'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    #Line below is code for interactive filtering options
    html.Hr(),
    html.Div([
        #Implemented dropdown options for data filtering
        dcc.Dropdown(
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'All', 'value': 'all'}
            ],
    value='all', #Defaults to unfiltered data
    id='rescue-dropdown'
        )
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        #Edited data table to make it more user-friendly for client
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0], #Initializing the selected row to 0 as default to initialize geolocation map
        page_action="native",
        page_current=0,
        page_size=30
    ),                 
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])


In [ ]:
#############################################
# Interaction Between Components / Controller
#############################################

#Changed first output argument from dropdown-output-container to hidden-div
    
@app.callback(Output('datatable-id','data'),
              [Input('rescue-dropdown', 'value')])
def update_dashboard(filter_type):
    #List acceptable dog breeds per type of rescue dog
    water_breeds = ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']
    mountain_breeds = ['German Shepard', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']
    disaster_breeds = ['Doberman Pinscher', 'German Shepard', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
    
    #Filter out database results depending on item selected from dropdown menu
    if filter_type == 'water':
        df = pd.DataFrame.from_records(db.read({"breed": {"$in": water_breeds}, "sex_upon_outcome": "Intact Female",
                                               "age_upon_outcome_in_weeks":{"$gte":26},
                                               "age_upon_outcome_in_weeks":{"$lte":156}}))
        
    elif filter_type == 'mountain':
        df = pd.DataFrame.from_records(db.read({"breed": {"$in": mountain_breeds}, "sex_upon_outcome": "Intact Male",
                                               "age_upon_outcome_in_weeks":{"$gte":26},
                                               "age_upon_outcome_in_weeks":{"$lte":156}}))
        
    elif filter_type == 'disaster':
        df = pd.DataFrame.from_records(db.read({"breed": {"$in": water_breeds}, "sex_upon_outcome": "Intact Male",
                                               "age_upon_outcome_in_weeks":{"$gte":20},
                                               "age_upon_outcome_in_weeks":{"$lte":300}}))
        
    else:
        df = pd.DataFrame.from_records(db.read({}))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]    
    data=df.to_dict('records')   
      
    return (data,columns)
#Callback for bar graph
@app.callback(
Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_graphs(viewData):
    breed_data = pd.DataFrame.from_dict(viewData)
    #Here I chose to make a bar chart
    fig = px.bar(breed_data, x='breed', title='Breed quantity')
    return [
        dcc.Graph(   
            figure = fig
        )    
    ]
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]
app.run_server(debug=True)